In [ ]:
# imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline,interp1d
from scipy.optimize import fsolve, minimize,minimize_scalar,root_scalar
import time
%matplotlib inline 

In [ ]:
# parameters
gamma = 0.5
beta = 0.975 # discount factor
r = 0.02 # interest rate
w = 1 # wage
T = 200 #simulation periods
convergance_tolerance = 1e-6
max_iterations = 2000
a_borrow = 5
# model parameters
egam = 1 - 1 / gamma
# grid parameters
a_l = -w / r
a_u = w / r
a_grid = np.linspace(a_l, a_u, 501)
a_l_borrowing = max(a_l, -a_borrow)


In [ ]:
# utility function
def utility(c):
    return np.where(c <= 0, -1e10, c**(1 - 1/gamma) / (1 - 1/gamma))

def marginal_utility(c):
    return np.where(c <= 0, 1e10, c ** (-1 / gamma))

def inv_marginal_utility(mu):
    return np.where(mu <= 0, 1e-10, mu ** (-gamma))

In [ ]:
# 8.4

# solve using value function iteration

def solve_vfi_min():
    V = np.zeros_like(a_grid) # initialize value function
    policy = np.zeros_like(a_grid) # initialize policy function

    for it in range(1000):
        V_new = np.zeros_like(V)
        V_interp = interp1d(a_grid, V, fill_value='extrapolate')
        for i, a in enumerate(a_grid):
            # define bellman RHS: for fixed a, find best c
            def bellman_objective(c):
                a_prime = (1 + r) * a + w - c
                return - (utility(c) + beta * V_interp(a_prime))  # negative for minimization

            upper_bound = a + w
            if upper_bound <= 1e-4:
                V_new[i] = -1e10
                policy[i] = 1e-4
                continue

            # minimize the negative of bellamn RHS
            res = minimize_scalar(bellman_objective, bounds=(1e-4, a + w), method='bounded')
            V_new[i] = -res.fun
            policy[i] = res.x # optimal consumption
        
        # check for convergence
        if np.max(np.abs(V - V_new)) < 1e-5:
            print(f"Minimization VFI converged in {it+1} iterations.")
            break
        V = V_new.copy()

    return V, policy

In [ ]:
# 8.5

# solving using Euler equation root-finding
def solve_vfi_root(a_grid):
    V = np.zeros_like(a_grid)
    policy = np.zeros_like(a_grid)
    for it in range(1000):
        V_new = np.zeros_like(V)
        V_func = interp1d(a_grid, V, fill_value="extrapolate")
        for i, a in enumerate(a_grid):
            def euler_residual(c):
                a_prime = (1 + r)*a + w - c
                return marginal_utility(c) - beta * (1 + r) * marginal_utility(V_func(a_prime))
            try:
                sol = root_scalar(euler_residual, bracket=[1e-4, a + w - 1e-4], method='bisect')
                c_opt = sol.root 
            except:
                c_opt = 1e-4

            a_prime = (1 + r)*a + w - c_opt
            policy[i] = c_opt
            V_new[i] = utility(c_opt) + beta * V_func(a_prime)
        if np.max(np.abs(V - V_new)) < 1e-5:
            print(f"Root-finding VFI converged in {it+1} iterations.")
            return V_new, policy
        V = V_new.copy()
    return V, policy

In [ ]:
# 8.6

# solving using endogeous grid method
def solve_egm(a_grid):
    V = np.zeros_like(a_grid)
    policy = np.zeros_like(a_grid)

    for it in range(1000):
        V_func = interp1d(a_grid, V, fill_value='extrapolate')
        V_prime = np.gradient(V, a_grid)
        Vp_func = interp1d(a_grid, V_prime, fill_value="extrapolate")

         # get optimal c for each a' using Euler equation inversion
        c_new = inv_marginal_utility(beta * (1 + r) * Vp_func(a_grid))
        a_new = (a_grid - w + c_new) / (1 + r)

        # remove invalid values
        valid = (a_new >= a_grid[0]) & (a_new <= a_grid[-1])
        a_new, c_new = a_new[valid], c_new[valid]

        # interpolate policy onto exogenous grid
        policy_func = interp1d(a_new, c_new, fill_value="extrapolate")
        policy = policy_func(a_grid)

        # update value function
        V_new = utility(policy) + beta * V_func((1 + r) * a_grid + w - policy)


        if np.max(np.abs(V - V_new)) < 1e-5:
                print(f"EGM converged in {it+1} iterations.")
                return V_new, policy
        V = V_new.copy()
        
    return V, policy

In [ ]:
# run all three methods and compare

V_min, policy_min = solve_vfi_min()
V_root, policy_root = solve_vfi_root(a_grid)
V_egm, policy_egm = solve_egm(a_grid)

# plot the policy and value functions
plt.figure(figsize=(14, 6))

# policy function comparison
plt.subplot(1, 2, 1)
plt.plot(a_grid, policy_min, label='VFI (minimization)')
plt.plot(a_grid, policy_root, '--', label='Root-Finding (8.5)')
plt.plot(a_grid, policy_egm, ':', label='EGM (8.6)')
plt.title("Policy Function Comparison")
plt.xlabel("Assets")
plt.ylabel("Consumption")
plt.legend()

# value function comparison
plt.subplot(1, 2, 2)
plt.plot(a_grid, V_min, label='Minimization (8.4)')
plt.plot(a_grid, V_root, '--', label='Root-Finding (8.5)')
plt.plot(a_grid, V_egm, ':', label='EGM (8.6)')
plt.title("Value Function Comparison")
plt.xlabel("Assets a")
plt.ylabel("Value V(a)")
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot([1, 2, 3], [1, 4, 9])
plt.title("Test plot")
plt.show()